In [3]:
import time
import pandas as pd
import numpy as np
from IPython.display import display

CITY_DATA = { 'chicago': 'chicago.csv',
              'new york city': 'new_york_city.csv',
              'washington': 'washington.csv' }
MONTHS = ['january', 'february', 'march', 'april', 'may', 'june', 'none']

DAYS =['monday', 'tuesday', 'wednesday', 'thursday',
        'friday', 'saturday', 'sunday', 'none']

def dayint_tostring(num):
    """
    This function takes an integer produced by the datetime module that
    represents a week day and returns the name of that week day as a string
    """
    mapp=['Monday', 'Tuesday', 'Wednesday', 'Thursday',
        'Friday', 'Saturday', 'Sunday']
    return mapp[num]

def monthint_tostring(num):
    """
    This function takes an integer produced by the datetime module
    that represents a month and returns the name of that month as a string
    """
    mapp=['January', 'February', 'March', 'April', 'May', 'June']
    return mapp[num-1]

In [4]:
def get_filters():
    """
    Asks user to specify a city, month, and day to analyze.

    Returns:
        (str) city - name of the city to analyze
        (str) month - name of the month to filter by, or "all" to apply no month filter
        (str) day - name of the day of week to filter by, or "all" to apply no day filter
        (book) True/False - whether the program should continue (True) or the user requested exit (False)
    """
    
    global CITY_DATA
    global MONTHS
    global DAYS
    print('Hello! Let\'s explore some US bikeshare data! Type exit anytime to shut down the program.')
        
    city = input('Which city would you like to see data for? Type "Chicago", "New York City, or "Washington".\n').lower().strip(' " \'')
    while True:
        if city == 'exit':
            return 0, 0, 0, False
        elif city in CITY_DATA:
            break
        else:
            city = input('This is not a valid city. \nType "Chicago", "New York City, "Washington", or "exit" to quit.\n').lower().strip(' " \'')
    
    month = input('To filter data by month, choose from (January, Febuary, March, April, May, June), or type none to ignore filter.\n').lower().strip(' " \'')
    while True:
        if month == 'exit':
            return 0, 0, 0, False
        elif month in MONTHS:
            break
        else:
            month = input('Invalid input. \nChoose from (January, Febuary, March, April, May, June) to filter by month, none for no filters, or "exit" to quit.\n').lower().strip(' " \'')
    
    day = input('To filter data by weekday, choose from (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday), or type none to ignore filter\n').lower().strip(' " \'')
    while True:
        if day == 'exit':
            return 0, 0, 0, False
        elif day in DAYS:
            break
        else:
            day = input('Invalid input. \nChoose from (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday) to filter by weekday, none for no filters, or "exit" to quit.\n').lower().strip(' " \'')

    print('-'*40)
    return city, month.title(), day.title(), True


def load_data(city, month, day):
    """
    Loads data for the specified city and filters by month and day if applicable.

    Args:
        (str) city - name of the city to analyze
        (str) month - name of the month to filter by, or none to apply no month filter
        (str) day - name of the day of week to filter by, or none to apply no day filter
    Returns:
        df - Pandas DataFrame containing city data filtered by month and day
    """
    
    global CITY_DATA
    city = CITY_DATA[city]
    
    df = pd.read_csv(city)
    df['Start Time'] = pd.to_datetime(df['Start Time'])
    df['Day of Week'] = df['Start Time'].dt.weekday
    df['Day of Week'] = df['Day of Week'].apply(dayint_tostring)
    df['Month'] = df['Start Time'].dt.month
    df['Month'] = df['Month'].apply(monthint_tostring)
    df['Hour'] = df['Start Time'].dt.hour

    df = df.drop(['Unnamed: 0'],axis=1)
    
    if month != 'None':
        df = df[df['Month'] ==month]
    
    if day != 'None':
        df = df[df['Day of Week'] ==day]
        
    print('Total trips in specified time is {}.'.format(df.shape[0]))
    return df



def time_stats(df):
    """Displays statistics on the most frequent times of travel."""

    print('\nCalculating The Most Frequent Times of Travel...\n')
    start_time = time.time()
    
    if len(df['Month'].unique()) > 1:
        highest = df['Month'].value_counts().idxmax()
        count = df['Month'].value_counts().max()
        print('The most popular month is {} with a total of {} rides.'.format(highest, count))

    if len(df['Day of Week'].unique()) > 1:
        highest = df['Day of Week'].value_counts().idxmax()
        count = df['Day of Week'].value_counts().max()
        print('The most popular day is {} with a total of {} rides.'.format(highest, count))
    
    highest = df['Hour'].value_counts().idxmax()
    count = df['Hour'].value_counts().max()
    
    if highest //12 ==0:
        clock = 'AM'
        if highest ==0:
            highest = 12
    else:
        clock = 'PM'
        if highest != 12:
            highest = highest % 12
    print('The most popular hour is {} {} with a total of {} rides.'.format(highest, clock, count))


    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)


def station_stats(df):
    """Displays statistics on the most popular stations and trip."""

    print('\nCalculating The Most Popular Stations and Trip...\n')
    start_time = time.time()
    # display most commonly used start station
    highest = df['Start Station'].value_counts().idxmax()
    count = df['Start Station'].value_counts().max()
    print('The most popular starting station is {} with a total of {} rides.'.format(highest, count))
   

    # display most commonly used end station
    highest = df['End Station'].value_counts().idxmax()
    count = df['End Station'].value_counts().max()
    print('The most popular ending station is {} with a total of {} rides.'.format(highest, count))

    # display most frequent combination of start station and end station trip
    src, dest = df[['Start Station','End Station']].value_counts().idxmax()
    count = df[['Start Station','End Station']].value_counts().max()
    print('The most popular trip starts from {} and ends on {} with a total of {} trips.'.format(src, dest, count))
    
    
    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)


def trip_duration_stats(df):
    """Displays statistics on the total and average trip duration."""

    print('\nCalculating Trip Duration...\n')
    start_time = time.time()

    # display total travel time
    sumTT= df['Trip Duration'].sum()
    print('Total trip duration is {} seconds, or {} hours.'.format(sumTT, sumTT//3600))

    # display mean travel time
    meanTT= df['Trip Duration'].mean()
    print('Average trip duration is {} seconds, or {} minutes.'.format(meanTT, meanTT//60))
    
    
    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)


def user_stats(df):
    """Displays statistics on bikeshare users."""

    print('\nCalculating User Stats...\n')
    start_time = time.time()

    print('Out of a total of {} users:'.format(df.shape[0]))
    
    if 'User Type' in df.columns:
        for UserType in df['User Type'].unique():
            countoftype = df[df['User Type']==UserType].shape[0]
            print('There are {} users of type {}.'.format(countoftype, UserType))
    
    # Display counts of gender
    if 'Gender' in df.columns:
        print('\nOut of a total of {} Subscribers:'.format(df[df['User Type']=='Subscriber'].shape[0]))
        for gender in df['Gender'].unique():
            if type(gender) != str:
                continue
            countofgen = df[df['Gender']==gender].shape[0]
            print('There are {} users who are {}.'.format(countofgen, gender))
    # Display earliest, most recent, and most common year of birth
    if 'Birth Year' in df.columns:
        dob = df['Birth Year']
        print('\nThe earliest year of birth is {}, the most recent is {}, and the most common is {}.'.format(dob.min(), dob.max(), dob.mode()[0]))
    

    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)

def ShowData(city):
    """
    This function takes the city name as input to load the raw data again.
    It then prints 5 rows iteratively whenever the user types 'yes'. The function exits when 
    any command other than 'yes' is given. 
    """
    Display = input('Would you like to see 5 lines of raw data? Type "yes" or the program will exit.').lower().strip(' " \'')
    global CITY_DATA
    city = CITY_DATA[city]
    df = pd.read_csv(city)
    x = 0
    maximumx = df.shape[0]
    while Display == 'yes':
        display(df.loc[range(x,x+5)])
        Display = input('Would you like to see more?')
        x=x+5
        if x+5 > maximumx:
            display(df.loc[range(x,maximumx)])
            print('No more data to display.')
            break
        

In [5]:
def main():
    while True:
        city, month, day, ContinueProgram = get_filters()
        if ContinueProgram:
            df = load_data(city, month, day)
            time_stats(df)
            station_stats(df)
            trip_duration_stats(df)
            user_stats(df)
            ShowData(city)

        restart = input('\nWould you like to restart? Type "yes" or the program will exit.\n').lower().strip(' " \'')
        if restart.lower() != 'yes':
            break


if __name__ == "__main__":
	main()

Hello! Let's explore some US bikeshare data! Type exit anytime to shut down the program.


Which city would you like to see data for? Type "Chicago", "New York City, or "Washington".
 chicago
To filter data by month, choose from (January, Febuary, March, April, May, June), or type none to ignore filter.
 none
To filter data by weekday, choose from (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday), or type none to ignore filter
 none


----------------------------------------
Total trips in specified time is 300000.

Calculating The Most Frequent Times of Travel...

The most popular month is June with a total of 98081 rides.
The most popular day is Tuesday with a total of 45912 rides.
The most popular hour is 5 PM with a total of 35992 rides.

This took 0.14092755317687988 seconds.
----------------------------------------

Calculating The Most Popular Stations and Trip...

The most popular starting station is Streeter Dr & Grand Ave with a total of 6911 rides.
The most popular ending station is Streeter Dr & Grand Ave with a total of 7512 rides.
The most popular trip starts from Lake Shore Dr & Monroe St and ends on Streeter Dr & Grand Ave with a total of 854 trips.

This took 0.36643242835998535 seconds.
----------------------------------------

Calculating Trip Duration...

Total trip duration is 280871787 seconds, or 78019 hours.
Average trip duration is 936.23929 seconds, or 15.0 minutes.

This took 0.00100088119

Would you like to see 5 lines of raw data? Type "yes" or the program will exit. yes


,Unnamed: 0,Start Time,End Time,Trip Duration,Start Station,End Station,User Type,Gender,Birth Year
0,1423854,2017-06-23 15:09:32,2017-06-23 15:14:53,321,Wood St & Hubbard St,Damen Ave & Chicago Ave,Subscriber,Male,1992.0
1,955915,2017-05-25 18:19:03,2017-05-25 18:45:53,1610,Theater on the Lake,Sheffield Ave & Waveland Ave,Subscriber,Female,1992.0
2,9031,2017-01-04 08:27:49,2017-01-04 08:34:45,416,May St & Taylor St,Wood St & Taylor St,Subscriber,Male,1981.0
3,304487,2017-03-06 13:49:38,2017-03-06 13:55:28,350,Christiana Ave & Lawrence Ave,St. Louis Ave & Balmoral Ave,Subscriber,Male,1986.0
4,45207,2017-01-17 14:53:07,2017-01-17 15:02:01,534,Clark St & Randolph St,Desplaines St & Jackson Blvd,Subscriber,Male,1975.0


...

Would you like to see more? yes


,Unnamed: 0,Start Time,End Time,Trip Duration,Start Station,End Station,User Type,Gender,Birth Year
15,958716,2017-05-25 22:59:33,2017-05-25 23:07:19,466,Clark St & Armitage Ave,Sheffield Ave & Wrightwood Ave,Subscriber,Female,1985.0
16,718598,2017-05-03 13:20:38,2017-05-03 13:31:13,635,Ada St & Washington Blvd,Daley Center Plaza,Subscriber,Male,1967.0
17,931608,2017-05-23 08:08:25,2017-05-23 08:11:09,164,Damen Ave & Cortland St,Marshfield Ave & Cortland St,Subscriber,Male,1990.0
18,1469705,2017-06-25 21:26:30,2017-06-25 21:33:44,434,State St & Pearson St,Clark St & Schiller St,Subscriber,Male,1987.0
19,475456,2017-04-08 11:37:55,2017-04-08 11:51:55,840,Adler Planetarium,Burnham Harbor,Customer,NaN,NaN


Would you like to see more? yes


,Unnamed: 0,Start Time,End Time,Trip Duration,Start Station,End Station,User Type,Gender,Birth Year
20,849468,2017-05-16 06:36:17,2017-05-16 06:59:34,1397,Michigan Ave & Oak St,Halsted St & Wrightwood Ave,Subscriber,Female,1992.0
21,1420915,2017-06-23 12:21:01,2017-06-23 12:32:54,713,Rush St & Cedar St,Halsted St & Willow St,Subscriber,Male,1985.0
22,161454,2017-02-12 15:24:29,2017-02-12 15:38:43,854,Clarendon Ave & Leland Ave,Lake Shore Dr & Belmont Ave,Subscriber,Male,1980.0
23,1413814,2017-06-22 19:14:37,2017-06-22 19:29:54,917,Bissell St & Armitage Ave,Orleans St & Chestnut St (NEXT Apts),Subscriber,Female,1989.0
24,717248,2017-05-03 09:41:11,2017-05-03 09:49:10,479,Wabash Ave & Roosevelt Rd,Halsted St & Roosevelt Rd,Subscriber,Male,1992.0


KeyboardInterrupt: Interrupted by user